In [1]:
import os
import json
import boto3
import sagemaker
import sagemaker.session

from sagemaker import utils
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput, CreateModelInput, TransformInput
from sagemaker.model import Model
from sagemaker.transformer import Transformer

from sagemaker.model_metrics import MetricsSource, ModelMetrics, FileSource
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.workflow.parameters import (
    ParameterBoolean,
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    CreateModelStep,
    TransformStep,
)
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.functions import Join
from sagemaker.model_monitor import DatasetFormat, model_monitoring
from sagemaker.clarify import BiasConfig, DataConfig, ModelConfig

In [2]:
region = sagemaker.Session().boto_region_name
sm_client = boto3.client("sagemaker")
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sm_client)
pipeline_session = PipelineSession()
prefix = "RegressionPredict"

role = sagemaker.get_execution_role()

In [17]:

baseline = ParameterString(
    name="baseline",
    default_value=f"s3://sm-dashboard-t/sagemaker-modelmonitor/train_headers/train_data_with_headers.csv",
)
test = ParameterString(
    name="test",
    default_value=f"s3://sm-dashboard-t/sagemaker-modelmonitor/test_data/test_data.csv",
)

image = '018079024734.dkr.ecr.us-east-1.amazonaws.com/evidently-image:latest'
script_processor = ScriptProcessor(
    image_uri='ubuntu-ev', # local image
    command=["python3"],
    instance_type="local",
    instance_count=1,

    role=role,
)

train_path = f"s3://sm-dashboard-t/sagemaker-modelmonitor/train_headers/train_data_with_headers.csv"
test_path = f"s3://sm-dashboard-t/sagemaker-modelmonitor/test_data/test_data.csv"


script_processor.run(
    code="evidently.py",
    arguments=["--baseline", train_path, "--test",test_path],
)


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating processing-job with name ubuntu-ev-2023-05-22-17-10-43-136
INFO:sagemaker.local.local_session:Starting processing job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-fekor:
    container_name: wdpc507eoj-algo-1-fekor
    entrypoint:
    - python3
    - /opt/ml/processing/input/code/evidently.py
    - --baseline
    - s3://sm-dashboard-t/sagemaker-modelmonitor/train_headers/train_data_with_headers.csv
    - --test
    - s3://sm-dashboard-t/sagemaker-modelmonitor/test_data/test_data.csv
    environment: []
    image: ubuntu-ev
    networks:
      sagemaker-local:
        aliases:
      

Creating wdpc507eoj-algo-1-fekor ... 
Creating wdpc507eoj-algo-1-fekor ... done
Attaching to wdpc507eoj-algo-1-fekor
wdpc507eoj-algo-1-fekor | /usr/share/python-wheels/requests-2.18.4-py2.py3-none-any.whl/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
wdpc507eoj-algo-1-fekor | Requirement already satisfied: evidently[test_suite] in /usr/local/lib/python3.6/dist-packages
wdpc507eoj-algo-1-fekor |   evidently 0.2.1 does not provide the extra 'test_suite'
wdpc507eoj-algo-1-fekor | Requirement already satisfied: dataclasses>=0.6 in /usr/local/lib/python3.6/dist-packages (from evidently[test_suite])
wdpc507eoj-algo-1-fekor | Requirement already satisfied: scikit-learn>=0.24.0 in /usr/local/lib/python3.6/dist-packages (from evidently[test_suite])
wdpc507eoj-algo-1-fekor | Requirement already satisfied: statsmodels>=0.12.2 in /usr/local/lib/python3.6/dist-packages (from evidently[test_suite])
wdpc507eoj-algo-1-fekor 

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpivrkfqpj/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit']